In [1]:
pwd

'/home/xiangyuan/Workspaces/TABLE_DETECTION_CTPN'

In [2]:
import model_detect_meta as meta
import model_detect_data as model_data

from model_detect_wrap import ModelDetect


import os
#
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' #使用 CPU
#os.environ['CUDA_VISIBLE_DEVICES'] = '0,1' # 使用 GPU 0，1
#


#
model = ModelDetect()
#



# data
print('loading data ...')
data_train = model_data.get_files_with_ext(meta.dir_images_train, 'png')[:100]
data_valid = model_data.get_files_with_ext(meta.dir_images_valid, 'png')[:100]
print('load finished.')

loading data ...
load finished.


In [3]:
if not os.path.exists(meta.model_detect_dir): os.mkdir(meta.model_detect_dir)

In [4]:
model.create_graph_all(training = True)

Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead
graph defined for training


In [5]:
import os
import time
import random

from PIL import Image
import numpy as np

import tensorflow as tf
from tensorflow.python.framework import graph_util

import model_detect_def as model_def
import model_detect_meta as meta

import model_detect_data as model_data

In [6]:
def get_target_json_file(img_file):
    #
    pre_dir = os.path.abspath(os.path.dirname(img_file)+os.path.sep+"..")
    json_dir = os.path.join(pre_dir, 'labels')
    #
    filename = os.path.basename(img_file)
    arr_split = os.path.splitext(filename)
    filename = arr_split[0] + '.json'
    #
    json_file = os.path.join(json_dir, filename)
    #
    return json_file

In [7]:
with model.graph.as_default():
    saver = tf.train.Saver()
    with tf.Session(config = model.sess_config) as sess:
        tf.global_variables_initializer().run()
        sess.run(tf.assign(model.learning_rate, tf.constant(model.learning_rate_base, dtype=tf.float32)))
        # restore with saved data
        ckpt = tf.train.get_checkpoint_state(meta.model_detect_dir)
        #
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)                
        #
        print('begin to train ...')
        step = sess.run(model.global_step)
#         train_step_half = int(model.train_steps * 0.5)
#         train_step_quar = int(model.train_steps * 0.75)
        
#         while step < self.train_steps:
        for _ in range(1):
            img_file = random.choice(data_train)  # list image files
            if not os.path.exists(img_file):
                print('image_file: %s NOT exist' % img_file)
                continue
#             txt_file = model_data.get_target_txt_file(img_file)
            json_file = get_target_json_file(img_file)
            if not os.path.exists(json_file):
                print('label_file: %s NOT exist' % json_file)
                continue
            img_data, feat_size, target_cls, target_ver, target_hor = \
                    model_data.get_image_and_targets(img_file, json_file, meta.anchor_heights)

INFO:tensorflow:Restoring parameters from ./model_detect/model_detect-0
begin to train ...


In [20]:
img_data[0].shape, feat_size, target_cls.shape, target_ver.shape, target_hor.shape

((842, 596, 3), [68, 75], (68, 75, 8), (68, 75, 8), (68, 75, 8))

In [10]:
import model_detect_data

In [11]:
txt_list = model_detect_data.get_list_contents_use_json(json_file)

In [12]:
txt_list

In [13]:
!vim /home/xiangyuan/Workspaces/gopath/data/labels/681_26.json

<Workspaces/gopath/data/labels/681_26.json" [noeol] 18L, 168C[>c[gopath/data/labels/681_26.json"
 {
  "cols": 9,
  "rows": 3,
  "x0": 68,
  "x1": 533,
  "y0": 193,
  "y1": 317
 },
 {
  "cols": 2,
  "rows": 6,
  "x0": 68,
  "x1": 533,
  "y0": 673,
  "y1": 760
 }
]
~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               1,1全部输入  :quit<Enter>  退出 Vim1,1全部[]2

In [14]:
os.path.exists(json_file)

True

In [17]:
import json
bboxes = []
with open(json_file, 'r') as fp:
    data = json.load(fp)
    for boxes in data:
        resized_box = [round(boxes['x0'] ),
                       round(boxes['y0'] ),
                       round(boxes['x1'] ),
                       round(boxes['y1'] )]
        bboxes.append(resized_box)

In [18]:
bboxes

[[68, 193, 533, 317], [68, 673, 533, 760]]

In [8]:
json_file

'/home/xiangyuan/Workspaces/gopath/data/labels/681_26.json'

In [29]:
import json

In [31]:
bboxes = []

with open(json_file, 'r') as fp:
    data = json.load(fp)
    for boxes in data:
        resized_box = [round(charline_obj['x0'] * image_resize_ratio),
                       round(charline_obj['y0'] * image_resize_ratio),
                       round(charline_obj['x1'] * image_resize_ratio),
                       round(charline_obj['y1'] * image_resize_ratio)]
        bboxes.append(resized_box)

In [32]:
data

[{'cols': 9, 'rows': 3, 'x0': 69, 'x1': 534, 'y0': 107, 'y1': 173},
 {'cols': 5, 'rows': 4, 'x0': 107, 'x1': 516, 'y0': 214, 'y1': 295},
 {'cols': 5, 'rows': 4, 'x0': 107, 'x1': 516, 'y0': 355, 'y1': 437},
 {'cols': 3, 'rows': 6, 'x0': 104, 'x1': 508, 'y0': 556, 'y1': 719}]

In [ ]:
#
def get_list_contents(content_file):
    #
    contents = []
    #
    if not os.path.exists(content_file): return contents
    #
    with open(content_file, 'r') as fp:
        lines = fp.readlines()
    #
    for line in lines:
        arr_str = line.split('|')
        item = list(map(lambda x: int(x), arr_str[0].split('-')))
        #
        contents.append([item, arr_str[1]])
        #
    return contents